# Regression with Categorical Predictors - Part 1

This set of notes will explore using linear regression for a single predictor attribute that is categorical instead of continuous. To explore this first, let's explore some data. 

In [ ]:
library(tidyverse)
library(Lahman)
library(ggformula)

theme_set(theme_bw(base_size = 18))

career <- Batting %>%
  filter(AB > 100) %>%
  anti_join(Pitching, by = "playerID") %>%
  filter(yearID > 1990) %>%
  group_by(playerID, lgID) %>%
  summarise(H = sum(H), AB = sum(AB)) %>%
  mutate(average = H / AB)

career <- People %>%
  tbl_df() %>%
  dplyr::select(playerID, nameFirst, nameLast) %>%
  unite(name, nameFirst, nameLast, sep = " ") %>%
  inner_join(career, by = "playerID") %>%
  dplyr::select(-playerID)

head(career)

## Question

Suppose we are interested in the batting average of baseball players since 1990, that is, the average is:

$$
average = \frac{number\ of\ hits}{number\ of\ atbats}
$$

Let's first visualize this.

In [ ]:
gf_density(~ average, data = career) %>%
  gf_labs(x = "Batting Average")

What if we hypothesized that the batting average will differ based on the league that players played in. 

In [ ]:
gf_violin(lgID ~ average, data = career, fill = 'gray80', draw_quantiles = c('0.1', '0.5', '0.9')) %>%
  gf_labs(x = "Batting Average",
          y = "League")

The distributions seem similar, but what if we wanted to go a step further and estimate a model to explore if there are really differences or not. For example, suppose we were interested in:

$$ 
H_{0}: \mu_{NL} = \mu_{AL}
$$

What type of model could we use? What about linear regression?

## Linear Regression with Categorical Attributes

Since these notes are happening, you can assume it is possible. But how can a categorical attribute with categories rather than numbers be included in the linear regression model?

The answer is that they can't. We need a new representation of the categorical attribute, enter dummy or indicator coding. 

### Dummy/Indicator Coding

Suppose we use the following logic:

If NL, then give a value of 1, else give a value of 0. 

Does this give the same information as before?

| League ID | Dummy League ID |
|-----------|-----------------|
| AL        | 0               | 
| NL        | 1               |

What would this look like for the actual data?

In [ ]:
career <- career %>%
  mutate(league_dummy = ifelse(lgID == 'NL', 1, 0))

head(career, n = 10)

Now that there is a numeric attribute, these can be added into the linear regression model.

In [ ]:
average_lm <- lm(average ~ league_dummy, data = career)

broom::tidy(average_lm)

How are these terms interpreted now?

In [ ]:
df_stats(average ~ league_dummy, data = career, mean, sd, length)

In [ ]:
average_lm2 <- lm(average ~ lgID, data = career)

broom::tidy(average_lm2)

In [ ]:
t.test(average ~ lgID, data = career, var.equal = TRUE)

## Values other than 0/1

First, I want to build off of the first part of the notes on regression with categorical predictors. Before generalizing to more than two groups, let's first explore what happens when values other than 0/1 are used for the categorical attribute. The following three dummy/indicator attributes will be used:

1. 1 = NL, 0 = AL
2. 1 = NL, 2 = AL
3. 100 = NL, 0 = AL

Make some predictions about what you think will happen in the three separate regressions?

In [ ]:
library(tidyverse)
library(Lahman)
library(ggformula)

theme_set(theme_bw(base_size = 18))

career <- Batting %>%
  filter(AB > 100) %>%
  anti_join(Pitching, by = "playerID") %>%
  filter(yearID > 1990) %>%
  group_by(playerID, lgID) %>%
  summarise(H = sum(H), AB = sum(AB)) %>%
  mutate(average = H / AB)

career <- People %>%
  tbl_df() %>%
  dplyr::select(playerID, nameFirst, nameLast) %>%
  unite(name, nameFirst, nameLast, sep = " ") %>%
  inner_join(career, by = "playerID") %>%
  dplyr::select(-playerID)

career <- career %>%
  mutate(league_dummy = ifelse(lgID == 'NL', 1, 0),
         league_dummy_12 = ifelse(lgID == 'NL', 1, 2),
         league_dummy_100 = ifelse(lgID == 'NL', 100, 0))

head(career, n = 10)

In [ ]:
average_lm <- lm(average ~ league_dummy, data = career)

broom::tidy(average_lm)

In [ ]:
average_lm_12 <- lm(average ~ league_dummy_12, data = career)

broom::tidy(average_lm_12)

In [ ]:
average_lm_100 <- lm(average ~ league_dummy_100, data = career)

broom::tidy(average_lm_100)

Before moving to more than 2 groups, any thoughts on how we could run a one-sample t-test using a linear regression? For example, suppose this null hypothesis wanted to be explored.

$$
H_{0}: \mu_{BA} = .2
$$

$$ 
H_{A}: \mu_{BA} \neq .2
$$

## Generalize to more than 2 groups

The ability to use regression with categorical attributes of more than 2 groups is possible and an extension of the 2 groups model shown above. First, let's think about how we could represent three categories as numeric attributes. Suppose we had the following 4 categories of baseball players.

| Position | 
|:---------| 
| Outfield | 
| Infield  | 
| Catcher  |
| Designated Hitter  |

In [ ]:
library(GeomMLBStadiums)

ggplot() + 
  geom_mlb_stadium(stadium_segments = "all") + 
  facet_wrap(~team) + 
  coord_fixed() + 
  theme_void()

In [ ]:
library(tidyverse)
library(Lahman)
library(ggformula)

theme_set(theme_bw(base_size = 18))

career <- Batting %>%
  filter(AB > 100) %>%
  anti_join(Pitching, by = "playerID") %>%
  filter(yearID > 1990) %>%
  group_by(playerID, lgID) %>%
  summarise(H = sum(H), AB = sum(AB)) %>%
  mutate(average = H / AB)

career <- Appearances %>%
  filter(yearID > 1990) %>%
  select(-GS, -G_ph, -G_pr, -G_batting, -G_defense, -G_p, -G_lf, -G_cf, -G_rf) %>%
  rowwise() %>%
  mutate(g_inf = sum(c_across(G_1b:G_ss))) %>%
  select(-G_1b, -G_2b, -G_3b, -G_ss) %>%
  group_by(playerID, lgID) %>%
  summarise(catcher = sum(G_c),
            outfield = sum(G_of),
            dh = sum(G_dh),
            infield = sum(g_inf),
            total_games = sum(G_all)) %>%
  pivot_longer(catcher:infield,
               names_to = "position") %>%
  filter(value > 0) %>%
  group_by(playerID, lgID) %>%
  slice_max(value) %>%
  select(playerID, lgID, position) %>%
  inner_join(career)

career <- People %>%
  tbl_df() %>%
  dplyr::select(playerID, nameFirst, nameLast) %>%
  unite(name, nameFirst, nameLast, sep = " ") %>%
  inner_join(career, by = "playerID")

career <- career %>%
  mutate(league_dummy = ifelse(lgID == 'NL', 1, 0))

count(career, position)

In [ ]:
gf_violin(position ~ average, data = career, fill = 'gray85', draw_quantiles = c(0.1, 0.5, 0.9)) %>%
  gf_labs(x = "Batting Average",
          y = "")

In [ ]:
career <- career %>%
  mutate(outfield = ifelse(position == 'outfield', 1, 0),
         infield = ifelse(position == 'infield', 1, 0),
         catcher = ifelse(position == 'catcher', 1, 0))

head(career)

In [ ]:
position_lm <- lm(average ~ 1 + outfield + infield + catcher, data = career)

broom::tidy(position_lm)

In [ ]:
df_stats(average ~ position, data = career, mean)